In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer


userA = pd.read_csv("/Desktop/Dataset/userA_chats.csv")
userB = pd.read_csv("/Desktop/Dataset/userB_chats.csv")


conversation = pd.DataFrame({
    'input': userB['message'],    
    'target': userA['message']   
})


train_df, test_df = train_test_split(conversation, test_size=0.1, random_state=42)


tokenizer = AutoTokenizer.from_pretrained("gpt2")


def tokenize_data(df):
    return tokenizer(
        df['input'].tolist(), df['target'].tolist(),
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

train_encodings = tokenize_data(train_df)
test_encodings = tokenize_data(test_df)

In [ ]:
import torch
from transformers import GPT2LMHeadModel


model = GPT2LMHeadModel.from_pretrained("gpt2")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ChatDataset(Dataset):
    def _init_(self, encodings):
        self.input_ids = encodings['input_ids']
        self.attention_mask = encodings['attention_mask']
        self.labels = encodings['input_ids']  # For GPT2 LM

    def _len_(self):
        return len(self.input_ids)

    def _getitem_(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

train_dataset = ChatDataset(train_encodings)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)


from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)


model.train()
for epoch in range(2):  # Keep epochs low for demo
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {loss.item()}")

In [ ]:
def generate_reply(userB_msg, max_length=50):
    model.eval()
    inputs = tokenizer(userB_msg, return_tensors="pt").to(device)
    reply_ids = model.generate(**inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    reply = tokenizer.decode(reply_ids[0], skip_special_tokens=True)
    return reply


userB_input = "Hey, are you coming to the party tonight?"
print("User A Reply:", generate_reply(userB_input))

In [ ]:
from nltk.translate.bleu_score import sentence_bleu


bleu_scores = []
for i in range(len(test_df)):
    ref = [test_df.iloc[i]['target'].split()]
    pred = generate_reply(test_df.iloc[i]['input']).split()
    bleu_scores.append(sentence_bleu(ref, pred))

print("Average BLEU:", sum(bleu_scores)/len(bleu_scores))

In [ ]:
import joblib
joblib.dump(model, "Model.joblib")